# Passing TDs - Draftkings

This is the url from the network tab: https://sportsbook-nash.draftkings.com/api/sportscontent/dkuswatl/v1/leagues/88808/categories/1000/subcategories/15987

Lets try to access it

In [ ]:
import requests

# The URL of the API endpoint
url = "https://sportsbook-nash.draftkings.com/api/sportscontent/dkuswatl/v1/leagues/88808/categories/1000/subcategories/15987"

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,ja;q=0.8,es;q=0.7",
    "Origin": "https://sportsbook.draftkings.com",
    "Referer": "https://sportsbook-nash.draftkings.com/",
    "Sec-Ch-Ua": '"Chromium";v="130", "Google Chrome";v="130", "Not:A_Brand";v="99"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "Windows",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
}


# Send a GET request to the URL
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON data
    data = response.json()
    print(data)  # Display the JSON data
else:
    print(f"Failed to retrieve data: {response.status_code}")


It works! This is passing TDs. Lets try predict Joe Burrow's passing TDs (no devigging)

First lets get the odds for all of Burrow's passing TDs

In [ ]:
def parse_qb_passing_tds_json(name: str):
  selections = data["selections"]
  odds = {}
  for s in selections:
    participants = s["participants"]
    for p in participants:
      if p["name"] == name:
        label = s["label"]
        odds[label] = s["displayOdds"]["american"]

  return odds

In [ ]:
burrow_odds = parse_qb_passing_tds_json("Joe Burrow")
burrow_odds

Great! Now we can convert these to implied probabilities

In [ ]:
def convert_to_implied_probability(odds):
  positive = odds[0] == '+'
  odds = int(odds[1:])
  if positive:
    return 100 / (odds + 100)
  else:
    return odds / (odds + 100)

In [ ]:
def convert_odds_dict_to_implied_probability(oddsDict, atLeast: int = 0):
  implied_probabilities = {tds: convert_to_implied_probability(odds) for tds, odds in oddsDict.items()}
  implied_probabilities[str(atLeast) + '+'] = 1.00 # 100% chance to for at least 0 TDs
  return implied_probabilities

In [ ]:
burrow_probabilities = convert_odds_dict_to_implied_probability(burrow_odds)
burrow_probabilities

From here, we can calculate the probabilities that Burrow gets exactly 0, 1, 2, 3, and 4+ TDs

In [ ]:
def convert_cumulative_to_exact(cumulative_probabilities):
  cumulative_probabilities = { int(k[0]):v for k, v in cumulative_probabilities.items() }
  exact_probabilities = {}
  upper_td_range = max(cumulative_probabilities) # some have 4+ td props, some have only 3
  for i in range(0, upper_td_range):
    exact_probabilities[i] = cumulative_probabilities[i] - cumulative_probabilities[i + 1]

  exact_probabilities[upper_td_range] = cumulative_probabilities[upper_td_range]
  return exact_probabilities

In [ ]:
exact_burrow_td_probabilities = convert_cumulative_to_exact(burrow_probabilities)
exact_burrow_td_probabilities

Finally, we can find the expected number of passing TDs by multiplying to find the EV

In [ ]:
def find_expected_value(exact_probabilities):
  ev = 0
  for k,v in exact_probabilities.items():
    ev += (k * v)

  return ev

In [ ]:
burrow_expected_tds = find_expected_value(exact_burrow_td_probabilities)
burrow_expected_tds

We can also calculate the amount of expected fantasy points!

In [ ]:
def find_expected_fantasy_points(statistic_value, fantasy_weight):
  return statistic_value * fantasy_weight

In [ ]:
find_expected_fantasy_points(burrow_expected_tds, fantasy_weight=4)

We expect Joe Burrow to get 7.88 fantasy points based on just passing TDs

Lets make this whole process into one simplified function (and try it out!)

In [ ]:
def find_expected_passing_tds(name: str):
  qb_passing_td_odds = parse_qb_passing_tds_json(name)
  implied_probabilities = convert_odds_dict_to_implied_probability(qb_passing_td_odds)
  exact_qb_td_probabilities = convert_cumulative_to_exact(implied_probabilities)
  return find_expected_value(exact_qb_td_probabilities)

In [ ]:
find_expected_passing_tds("Joe Burrow")

Lets do this for every QB that has props!

In [ ]:
def get_all_expected_passing_tds():
  expected_tds = {}
  for s in data["selections"]:
    participants = s["participants"]
    for p in participants:
      if p["name"] not in expected_tds:
        expected_tds[p["name"]] = find_expected_passing_tds(p["name"])

  return expected_tds

In [ ]:
get_all_expected_passing_tds()

Find the expected fantasy points from passing TDs too:

In [ ]:
def get_all_expected_passing_td_fantasy_points():
  expected_tds = get_all_expected_passing_tds()
  expected_fantasy_points = { player:find_expected_fantasy_points(tds, fantasy_weight=4) for player, tds in expected_tds.items() }
  return expected_fantasy_points

In [ ]:
fantasy_points_passing_tds = get_all_expected_passing_td_fantasy_points()
sorted(fantasy_points_passing_tds.items(), key=lambda x: x[1], reverse=True)